# RESOLVE-G3 Groups Tutorial

Author: Zack Hutchens<br>
Date: February 23, 2021

This Jupyter tutorial provides a demonstration of the G3 group finding technique, which is presented fully at https://github.com/zhutchens1/g3groups. In the following cells, steps 1-4 mirror the outline of the git README, replicating the group catalog within this notebook. Following the group finding, we show some simple examples of plots/figures to demonstrate the G3 group data columns.

## Step 0: Data & Group Finding Selection
First, let's read in the RESOLVE and ECO data. The catalogs read-in below can be found on the RESOLVE-G3 slack (https://resolve-g3.slack.com). While these prototype catalogs already contain the G3 group information, we will reproduce the results here to demonstrate our algorithm. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from scipy.stats import binned_statistic
from smoothedbootstrap import smoothedbootstrap as sbs
import foftools as fof
import iterativecombination as ic

resall = np.genfromtxt("RESOLVEdata_G3catalog_luminosity.csv", delimiter=",", dtype=None, names=True, encoding=None)
ecoall = np.genfromtxt("ECOdata_G3catalog_luminosity.csv", delimiter=",", dtype=None, names=True, encoding=None)
ecovolume = 192351.36 # Mpc^3 (h=1)

The RESOLVE survey comprises two equatorial strips, the A- and B- semester, for which the A-semester is a subset of the larger ECO catalog (see https://resolve.astro.unc.edu for details). For this reason, our group finding procedure will be applied to ECO and RESOLVE-B separately, which allows us to (a) extract RESOLVE-A groups from ECO and (b) use ECO groups to control for cosmic variance effects in the smaller-volume RESOLVE-B. Thus, in the next cell, we will refine our datasets to reflect division of the A/B semesters. Throughout this tutorial, there will be references to a "RESOLVE-B analog" dataset, which is a version of ECO down to -17.0 (not -17.33) that enables us to extract group finding information for the denser, more-complete RESOLVE-B.

In [ ]:
# ECO 
eco = ecoall
resb = resall[(resall['f_b']==1)]

In [ ]:
# make distinct arrays + prepare arrays to store group ID's (*g3grp)
ecosz = len(eco)
econame = np.array(eco['name'])
ecoresname = np.array(eco['resname']) # RESOLVE name in ECO catalog
ecoradeg = np.array(eco['radeg'])
ecodedeg = np.array(eco['dedeg'])
ecocz = np.array(eco['cz'])
ecoabsrmag = np.array(eco['absrmag'])
ecog3grp = np.full(ecosz, -99.)
ecog3logmh = np.full(ecosz,-99.) # abundance-matched halo mass 

resbsz = int(len(resb))
resbname = np.array(resb['name'])
resbradeg = np.array(resb['radeg'])
resbdedeg = np.array(resb['dedeg'])
resbcz = np.array(resb['cz'])
resbabsrmag = np.array(resb['absrmag'])
resbg3grp = np.full(resbsz, -99.)
resbg3logmh = np.full(resbsz, -99.)

resbana_g3grp = np.full(ecosz,-99.) # for RESOLVE-B analog

## Step 1: Finding Giant-Only Cores of Groups
Now that the data is stored, we can begin working on group finding. As described in the README, the first step is identify "group cores" in RESOLVE-B and ECO using giant galaxies. These group cores are defined by friends-of-friends using the linking constants recommended by Duarte & Mamon (2014). For the luminosity-selected catalog, we divide dwarfs and giants at $M_r = -19.4$.

The beta catalogs provided in the slack are based on an adaptive linking strategy in Step 1. To see how the giant-only groups change if the linking lengths are fixed, set `ADAPTIVE_LINKING` to zero in the next cell. Run the next two cells to apply FoF to giant galaxies and see the multiplicity function for giants.

In [ ]:
ADAPTIVE_LINKING = 1

# create adaptive strategy from ECO giant galaxies
ecogiantsel = (ecoabsrmag<=-19.4)
s0 = (ecovolume/len(ecoabsrmag[ecogiantsel]))**(1/3.)
ecogiantmags = ecoabsrmag[ecogiantsel]
ecogiantsepdata = np.array([(192351.36/len(ecogiantmags[ecogiantmags<=Mr]))**(1/3.) for Mr in ecogiantmags])
ecogiantsepdata = ecogiantsepdata*s0/np.median(ecogiantsepdata)
poptsfit, pcovsfit = curve_fit(fof.sepmodel, ecogiantmags, ecogiantsepdata) 
meansepinterp = lambda x: fof.sepmodel(x, *poptsfit) 

# get adaptive separations for ECO, RESOLVE-B (smoothed by model fit)
ecogiantsep = meansepinterp(ecogiantmags)
resbgiantsel = (resbabsrmag<=-19.4)
resbgiantsep = meansepinterp(resbabsrmag[(resbgiantsel)])

ecolinking = ADAPTIVE_LINKING*(ecogiantsep)+(1-ADAPTIVE_LINKING)*s0
resblinking = ADAPTIVE_LINKING*(resbgiantsep)+(1-ADAPTIVE_LINKING)*s0

# run FoF group ID numbers for ECO (+ RESOLVE-B analogue) and RESOLVE-B
blos, bperp = 1.1, 0.07 # Duarte & Mamon (2014)
ecogiantfofid = fof.fast_fof(ecoradeg[ecogiantsel], ecodedeg[ecogiantsel], ecocz[ecogiantsel], bperp, blos,\
                             ecolinking)
ecog3grp[ecogiantsel] = ecogiantfofid
resbana_g3grp[ecogiantsel] = ecogiantfofid

resbgiantfofid = fof.fast_fof(resbradeg[resbgiantsel], resbdedeg[resbgiantsel], resbcz[resbgiantsel], bperp, blos,\
                              resblinking)
resbg3grp[resbgiantsel]=resbgiantfofid

In [ ]:
fig, (ax, ax1) = plt.subplots(ncols=2,figsize=(16,5))
tx=np.linspace(-24,-19.3,100)
ax.axhline(s0, label=r'Mean Separation of ECO Giant Galaxies, $s_0 = (V/N)^{1/3}$', color='k', linestyle='--')
ax.plot(tx, meansepinterp(tx), label='Model Fit')
ax.plot(ecogiantmags, ecogiantsepdata, 'k.', alpha=1, label=r'ECO Giant Galaxies ($M_r \leq -19.4$)')
ax.plot(resbabsrmag[resbgiantsel], resbgiantsep, 'r^', alpha=0.4, label=r'RESOLVE-B Giant Galaxies (interpolated, $M_r \leq -19.4$)')
ax.set_xlabel("Absolute $M_r$ of Giant Galaxy")
ax.set_ylabel(r"$s_i$ - Separation used for Galaxy $i$ in Giant-Only FoF [Mpc/h]")
ax.legend(loc='best')
ax.invert_xaxis()

binv = np.arange(0.5,300.5,3)
ax1.hist(fof.multiplicity_function(ecog3grp[ecog3grp!=-99.], return_by_galaxy=False), bins=binv, histtype='step', linewidth=3, label='ECO Giant-Only FoF Groups')
ax1.hist(fof.multiplicity_function(resbg3grp[resbg3grp!=-99.], return_by_galaxy=False), bins=binv, histtype='step', linewidth=1.5, hatch='\\', label='RESOLVE-B Giant-Only FoF Groups')
ax1.set_xlabel("Number of Giant Galaxies per Group")
ax1.set_ylabel("Number of Giant-Only FoF Groups")
ax1.set_yscale('log')
ax1.legend(loc='best')
ax1.set_xlim(0,80)
plt.show()

## Step 2: Associating Dwarf Galaxies to Giant-Only Groups
The next step in the algorithm is to search for dwarf galaxies near the giant-only FoF groups. The boundaries will be derived from model fits against group-$N$, which allows us to (i) avoid halo mass-based membership refinement and (ii) extrapolate boundaries to associate dwarfs around isolated giants. We'll need to: 

(a) Compute relative radii and peculiar velocities of giant galaxies within their groups<br>
(b) Compute medians of those quantities in group-N bins (with bootstrapped errors on medians)<br>
(c) Fit the medians to a logarithmic model<br>
(d) Scale the model to skirt the outermost galaxies in radius/velocity space and calibrate against abundance-matched halos<br>
(e) Associate dwarfs into groups based on fitted boundaries. This steps adds dwarf galaxies to the giant-only groups if they fall within the provided radius and velocity boundaries. If two giant-only groups are competing for a dwarf member, it goes to the group whose center is closest to the dwarf. <br>

In [ ]:
# (a)
ecogiantgrpra, ecogiantgrpdec, ecogiantgrpcz = fof.group_skycoords(ecoradeg[ecogiantsel], ecodedeg[ecogiantsel], ecocz[ecogiantsel], ecogiantfofid)
relvel = np.abs(ecogiantgrpcz - ecocz[ecogiantsel])
relprojdist = (ecogiantgrpcz + ecocz[ecogiantsel])/100. * ic.angular_separation(ecogiantgrpra, ecogiantgrpdec, ecoradeg[ecogiantsel], ecodedeg[ecogiantsel])/2.0
ecogiantgrpn = fof.multiplicity_function(ecogiantfofid, return_by_galaxy=True) # returns group N by galaxy.
uniqecogiantgrpn, uniqindex = np.unique(ecogiantgrpn, return_index=True)
keepcalsel = np.where(uniqecogiantgrpn>1)

In [ ]:
# (b)
median_relprojdist = np.array([np.median(relprojdist[np.where(ecogiantgrpn==sz)]) for sz in uniqecogiantgrpn[keepcalsel]])
median_relvel = np.array([np.median(relvel[np.where(ecogiantgrpn==sz)]) for sz in uniqecogiantgrpn[keepcalsel]])
rproj_median_error = np.std(np.array([sbs(relprojdist[np.where(ecogiantgrpn==sz)], 10000, np.median, kwargs=dict({'axis':1 })) for sz in uniqecogiantgrpn[keepcalsel]]), axis=1)
dvproj_median_error = np.std(np.array([sbs(relvel[np.where(ecogiantgrpn==sz)], 10000, np.median, kwargs=dict({'axis':1})) for sz in uniqecogiantgrpn[keepcalsel]]), axis=1)

In [ ]:
# (c)
poptrproj, jk = curve_fit(fof.giantmodel, uniqecogiantgrpn[keepcalsel], median_relprojdist, sigma=rproj_median_error, p0=[0.1, -2, 3, -0.1])
poptdvproj,jk = curve_fit(fof.giantmodel, uniqecogiantgrpn[keepcalsel], median_relvel, sigma=dvproj_median_error, p0=[160,6.5,45,-600]) 
rproj_boundary = lambda N: 3*fof.giantmodel(N, *poptrproj)
vproj_boundary = lambda N: 4.5*fof.giantmodel(N, *poptdvproj)

In [ ]:
# (d)

# Commented-out lines do halo abundance matching (frozen from this tutorial, ask Zack for more details)
#gihaloid, gilogmh, gir200, gihalovdisp = ic.HAMwrapper(ecoradeg[ecogiantsel], ecodedeg[ecogiantsel], ecocz[ecogiantsel], ecoabsrmag[ecogiantsel], ecog3grp[ecogiantsel],\
#                                      ecovolume, inputfilename=None, outputfilename=None)
#gihalorvir = (3*(10**gilogmh / fof.getmhoffset(200,337,1,1,6)) / (4*np.pi*337*0.3*2.77e11) )**(1/3.)
#gihalon = fof.multiplicity_function(np.sort(ecog3grp[ecogiantsel]), return_by_galaxy=False)

if True:
    fig, (ax,ax1)=plt.subplots(ncols=2, figsize=(15,4))
    sel = (ecogiantgrpn>1)
    #ax.scatter(gihalon, gihalovdisp, marker='D', color='purple', label=r'ECO HAM Velocity Dispersion')
    ax.plot(ecogiantgrpn[sel], relvel[sel], 'r.', alpha=0.2, label='ECO Giant Galaxies')
    ax.errorbar(uniqecogiantgrpn[keepcalsel], median_relvel, fmt='k^', label=r'$\Delta v_{\rm proj}$ (Median of $\Delta v_{\rm proj,\, gal}$)',yerr=dvproj_median_error)
    tx = np.linspace(1,max(ecogiantgrpn),1000)
    ax.plot(tx, fof.giantmodel(tx, *poptdvproj), label=r'$1\Delta v_{\rm proj}^{\rm fit}$')
    ax.plot(tx, 4.5*fof.giantmodel(tx, *poptdvproj), 'g',  label=r'$4.5\Delta v_{\rm proj}^{\rm fit}$', linestyle='-.')
    ax.set_xlabel("Number of Giant Members")
    ax.set_ylabel("Relative Velocity to Group Center [km/s]")
    ax.legend(loc='best')

    #ax1.scatter(gihalon, gihalorvir, marker='D', color='purple', label=r'ECO Group Virial Radii')
    ax1.plot(ecogiantgrpn[sel], relprojdist[sel], 'r.', alpha=0.2, label='ECO Giant Galaxies')
    ax1.errorbar(uniqecogiantgrpn[keepcalsel], median_relprojdist, fmt='k^', label=r'$R_{\rm proj}$ (Median of $R_{\rm proj,\, gal}$)',yerr=rproj_median_error)
    ax1.plot(tx, fof.giantmodel(tx, *poptrproj), label=r'$1R_{\rm proj}^{\rm fit}$')
    ax1.plot(tx, 3*fof.giantmodel(tx, *poptrproj), 'g', label=r'$3R_{\rm proj}^{\rm fit}$', linestyle='-.')
    ax1.set_xlabel("Number of Giant Members in Galaxy's Group")
    ax1.set_ylabel("Projected Distance from Giant to Group Center [Mpc/h]")
    ax1.legend(loc='best')
    plt.show()


In [ ]:
# (e)
ecodwarfsel = (ecoabsrmag>-19.4) & (ecoabsrmag<=-17.33) & (ecocz>2530) & (ecocz<7470)
resbdwarfsel = (resbabsrmag>-19.4) & (resbabsrmag<=-17.0) & (resbcz>4250) & (resbcz<7250)
resbana_dwarfsel = (ecoabsrmag>-19.4) & (ecoabsrmag<=-17.0) & (ecocz>2530) & (ecocz<7470)    

resbgiantgrpra, resbgiantgrpdec, resbgiantgrpcz = fof.group_skycoords(resbradeg[resbgiantsel], resbdedeg[resbgiantsel], resbcz[resbgiantsel], resbgiantfofid)
resbgiantgrpn = fof.multiplicity_function(resbgiantfofid, return_by_galaxy=True)
ecodwarfassocid, junk = fof.fast_faint_assoc(ecoradeg[ecodwarfsel],ecodedeg[ecodwarfsel],ecocz[ecodwarfsel],ecogiantgrpra,ecogiantgrpdec,ecogiantgrpcz,ecogiantfofid,\
                   rproj_boundary(ecogiantgrpn),vproj_boundary(ecogiantgrpn))
resbdwarfassocid, junk = fof.fast_faint_assoc(resbradeg[resbdwarfsel],resbdedeg[resbdwarfsel],resbcz[resbdwarfsel],resbgiantgrpra,resbgiantgrpdec,resbgiantgrpcz,resbgiantfofid,\
                   rproj_boundary(resbgiantgrpn),vproj_boundary(resbgiantgrpn))
    
resbana_dwarfassocid, junk = fof.fast_faint_assoc(ecoradeg[resbana_dwarfsel], ecodedeg[resbana_dwarfsel], ecocz[resbana_dwarfsel], ecogiantgrpra, ecogiantgrpdec, ecogiantgrpcz, ecogiantfofid,\
                                                    rproj_boundary(ecogiantgrpn), vproj_boundary(ecogiantgrpn))

ecog3grp[ecodwarfsel] = ecodwarfassocid
resbg3grp[resbdwarfsel] = resbdwarfassocid
resbana_g3grp[resbana_dwarfsel] = resbana_dwarfassocid

# Step 3: Finding Dwarf-Only Groups
Steps #1 and #2 miss any groups composed strictly of dwarf galaxies, since the method depends on having giant galaxies to associate around. Thus, the next step is to construct dwarf-only groups from the remaining ungrouped dwarfs. Our "iterative combination" procedure is detailed on the README page. In practice, the steps are to:

 (a) Select galaxies in "giant+dwarf" groups (i.e. that are NOT ungrouped dwarf galaxies)<br>
 (b) Compute the relative radii and peculiar velocities of "giant+dwarf" group members<br>
 (c) Bin these quantities and fit them against group-integrated luminosity<br>
 (d) Construct groups using iterative combination<br>
 

In [ ]:
# (a) galaxy selection
ecogdgrpn = fof.multiplicity_function(ecog3grp, return_by_galaxy=True)
ecogdsel = np.logical_not((ecogdgrpn==1) & (ecoabsrmag>-19.4) & (ecog3grp>0)) # select galaxies that AREN'T ungrouped dwarfs
ecogdsel = np.logical_not(np.logical_or(ecog3grp==-99., ((ecogdgrpn==1) & (ecoabsrmag>-19.4) & (ecoabsrmag<=-17.33))))

In [ ]:
# (b) compute relative radii + peculiar velocities
ecogdgrpra, ecogdgrpdec, ecogdgrpcz = fof.group_skycoords(ecoradeg[ecogdsel], ecodedeg[ecogdsel], ecocz[ecogdsel], ecog3grp[ecogdsel])
ecogdrelvel = np.abs(ecogdgrpcz - ecocz[ecogdsel])
ecogdrelprojdist = (ecogdgrpcz + ecocz[ecogdsel])/100. * ic.angular_separation(ecogdgrpra, ecogdgrpdec, ecoradeg[ecogdsel], ecodedeg[ecogdsel])/2.0
ecogdn = ecogdgrpn[ecogdsel]
ecogdtotalmag = ic.get_int_mag(ecoabsrmag[ecogdsel], ecog3grp[ecogdsel]) # group L_r

In [ ]:
# (c) bin quantities and curve fit
magbins=np.arange(-24,-19,0.25)
binsel = np.where(np.logical_and(ecogdn>1, ecogdtotalmag>-24))
gdmedianrproj, magbinedges, jk = binned_statistic(ecogdtotalmag[binsel], ecogdrelprojdist[binsel], lambda x:np.nanpercentile(x,99), bins=magbins)
gdmedianrelvel, jk, jk = binned_statistic(ecogdtotalmag[binsel], ecogdrelvel[binsel], lambda x: np.nanpercentile(x,99), bins=magbins)
nansel = np.isnan(gdmedianrproj)
if ADAPTIVE_LINKING:
    guess=None
else:
    guess=[1e-5, 0.4, 0.2, 1]
poptr, pcovr = curve_fit(ic.decayexp, magbinedges[:-1][~nansel], gdmedianrproj[~nansel], p0=guess)
poptv, pcovv = curve_fit(ic.decayexp, magbinedges[:-1][~nansel], gdmedianrelvel[~nansel], p0=[3e-5,4e-1,5e-03,1])

tx = np.linspace(-25,-17,100)
fig, (ax, ax1) = plt.subplots(ncols=2, figsize=(15,4))
ax.plot(ecogdtotalmag[binsel], ecogdrelprojdist[binsel], 'k.', alpha=0.2, label='ECO Galaxies in N>1 Giant+Dwarf Groups')
ax.plot(magbinedges[:-1], gdmedianrproj, 'r^', label='99th percentile in bin')
ax.plot(tx, ic.decayexp(tx,*poptr))
ax.set_xlabel(r"Integrated $M_r$ of Giant + Dwarf Members")
ax.set_ylabel("Projected Distance from Galaxy to Group Center [Mpc/h]")
ax.legend(loc='best')
ax.invert_xaxis()

ax1.plot(ecogdtotalmag[binsel], ecogdrelvel[binsel], 'k.', alpha=0.2, label='ECO Galaxies in N>1 Giant+Dwarf Groups')
ax1.plot(magbinedges[:-1], gdmedianrelvel,'r^',label='Medians')
ax1.plot(tx, ic.decayexp(tx, *poptv))
ax1.set_ylabel("Relative Velocity between Galaxy and Group Center")
ax1.set_xlabel(r"Integrated $M_r$ of Giant + Dwarf Members")
ax1.invert_xaxis()
ax1.legend(loc='best')
plt.show()

rproj_for_iteration = lambda M: ic.decayexp(M, *poptr)
vproj_for_iteration = lambda M: ic.decayexp(M, *poptv)

In [ ]:
# (d) identify dwarf-only groups using iterative combination
assert (ecog3grp[(ecoabsrmag<=-19.4) & (ecocz<7470) & (ecocz>2530)]!=-99.).all(), "Not all giants are grouped."

ecogrpnafterassoc = fof.multiplicity_function(ecog3grp, return_by_galaxy=True)
resbgrpnafterassoc = fof.multiplicity_function(resbg3grp, return_by_galaxy=True)
resbana_grpnafterassoc = fof.multiplicity_function(resbana_g3grp, return_by_galaxy=True)

eco_ungroupeddwarf_sel = (ecoabsrmag>-19.4) & (ecoabsrmag<=-17.33) & (ecocz<7470) & (ecocz>2530) & (ecogrpnafterassoc==1)
ecoitassocid = ic.iterative_combination(ecoradeg[eco_ungroupeddwarf_sel], ecodedeg[eco_ungroupeddwarf_sel], ecocz[eco_ungroupeddwarf_sel], ecoabsrmag[eco_ungroupeddwarf_sel],\
                rproj_for_iteration, vproj_for_iteration, starting_id=np.max(ecog3grp)+1, centermethod='arithmetic')
resb_ungroupeddwarf_sel = (resbabsrmag>-19.4) & (resbabsrmag<=-17.0) & (resbcz<7250) & (resbcz>4250) & (resbgrpnafterassoc==1)
resbitassocid = ic.iterative_combination(resbradeg[resb_ungroupeddwarf_sel], resbdedeg[resb_ungroupeddwarf_sel], resbcz[resb_ungroupeddwarf_sel], resbabsrmag[resb_ungroupeddwarf_sel],\
                rproj_for_iteration, vproj_for_iteration, starting_id=np.max(resbg3grp)+1, centermethod='arithmetic')
resbana_ungroupeddwarf_sel = (ecoabsrmag>-19.4) & (ecoabsrmag<=-17.0) & (ecocz<7470) & (ecocz>2530) & (resbana_grpnafterassoc==1)
resbana_itassocid = ic.iterative_combination(ecoradeg[resbana_ungroupeddwarf_sel], ecodedeg[resbana_ungroupeddwarf_sel], ecocz[resbana_ungroupeddwarf_sel], ecoabsrmag[resbana_ungroupeddwarf_sel],\
                    rproj_for_iteration, vproj_for_iteration, starting_id=np.max(resbana_g3grp)+1, centermethod='arithmetic')

ecog3grp[eco_ungroupeddwarf_sel] = ecoitassocid
resbg3grp[resb_ungroupeddwarf_sel] = resbitassocid
resbana_g3grp[resbana_ungroupeddwarf_sel] = resbana_itassocid

In [ ]:
plt.figure()
binv = np.arange(0.5,1200.5,3)
plt.hist(fof.multiplicity_function(ecog3grp[ecog3grp!=-99.], return_by_galaxy=False), bins=binv, log=True, label='ECO Groups', histtype='step', linewidth=3)
plt.hist(fof.multiplicity_function(resbg3grp[resbg3grp!=-99.], return_by_galaxy=False), bins=binv, log=True, label='RESOLVE-B Groups', histtype='step', hatch='\\')
plt.xlabel("Number of Giant + Dwarf Group Members")
plt.ylabel("Number of Groups")
plt.legend(loc='best')
plt.xlim(0,100)
plt.show()

## Step 4: Assigning Halo Masses
Although the motivation for this algorithm requires us to not use halo mass estimates for membership refine, we can still apply halo abundance matching to RESOLVE and ECO, since we know them to be highly-complete. HAM group masses are provided in the Slack catalogs, but are left-out of this tutorial for simplicity (e.g., complications due to python dependencies). 

## Finalizing the Group Catalog

Now that our group finding is complete in ECO and RESOLVE-B, we should calculate additional quantities using functions that are pre-built into my `foftools` package - such as the central flag, velocity dispersion, and group radii. The cells below compute these quantities for RESOLVE-B and ECO and store them in the original dataframes we read-in.

In [ ]:
ecog3grpngi = np.zeros(len(ecog3grp))
ecog3grpndw = np.zeros(len(ecog3grp))
for uid in np.unique(ecog3grp):
    grpsel = np.where(ecog3grp==uid)
    gisel = np.where(np.logical_and((ecog3grp==uid),(ecoabsrmag<=-19.4)))
    dwsel = np.where(np.logical_and((ecog3grp==uid), (ecoabsrmag>-19.4)))
    if len(gisel[0])>0.:
        ecog3grpngi[grpsel] = len(gisel[0])
    if len(dwsel[0])>0.:
        ecog3grpndw[grpsel] = len(dwsel[0])

ecog3grpradeg, ecog3grpdedeg, ecog3grpcz = fof.group_skycoords(ecoradeg, ecodedeg, ecocz, ecog3grp)
ecog3rproj = fof.get_grprproj_e17(ecoradeg, ecodedeg, ecocz, ecog3grp, h=0.7) / (ecog3grpcz/70.) * 206265 # in arcsec
ecog3fc = fof.get_central_flag(ecoabsrmag, ecog3grp)
ecog3router = fof.get_outermost_galradius(ecoradeg, ecodedeg, ecocz, ecog3grp) # in arcsec
junk, ecog3vdisp = fof.get_rproj_czdisp(ecoradeg, ecodedeg, ecocz, ecog3grp)

outofsample = (ecog3grp==-99.)
ecog3grpngi[outofsample]=-99.
ecog3grpndw[outofsample]=-99.
ecog3grpradeg[outofsample]=-99.
ecog3grpdedeg[outofsample]=-99.
ecog3grpcz[outofsample]=-99.
ecog3rproj[outofsample]=-99.
ecog3fc[outofsample]=-99.
ecog3router[outofsample]=-99.
ecog3vdisp[outofsample]=-99.

ecoall['g3grp_l']=ecog3grp
ecoall['g3grpngi_l']=ecog3grpngi
ecoall['g3grpndw_l']=ecog3grpndw
ecoall['g3grpradeg_l']=ecog3grpradeg
ecoall['g3grpdedeg_l']=ecog3grpdedeg
ecoall['g3grpcz_l']=ecog3grpcz
ecoall['g3rproj_l']=ecog3rproj
ecoall['g3fc_l']=ecog3fc
ecoall['g3router_l']=ecog3router
ecoall['g3vdisp_l']=ecog3vdisp

In [ ]:
resbg3grpngi = np.zeros(len(resbg3grp))
resbg3grpndw = np.zeros(len(resbg3grp))
for uid in np.unique(resbg3grp):
    grpsel = np.where(resbg3grp==uid)
    gisel = np.where(np.logical_and((resbg3grp==uid),(resbabsrmag<=-19.4)))
    dwsel = np.where(np.logical_and((resbg3grp==uid), (resbabsrmag>-19.4)))
    if len(gisel[0])>0.:
        resbg3grpngi[grpsel] = len(gisel[0])
    if len(dwsel[0])>0.:
        resbg3grpndw[grpsel] = len(dwsel[0])
resbg3grpradeg, resbg3grpdedeg, resbg3grpcz = fof.group_skycoords(resbradeg, resbdedeg, resbcz, resbg3grp)
resbg3rproj = fof.get_grprproj_e17(resbradeg, resbdedeg, resbcz, resbg3grp, h=0.7) / (resbg3grpcz/70.) * 206265 # in arcsec 
resbg3fc = fof.get_central_flag(resbabsrmag, resbg3grp)
resbg3router = fof.get_outermost_galradius(resbradeg, resbdedeg, resbcz, resbg3grp) # in arcsec
junk, resbg3vdisp = fof.get_rproj_czdisp(resbradeg, resbdedeg, resbcz, resbg3grp)

outofsample = (resbg3grp==-99.)
resbg3grpngi[outofsample]=-99.
resbg3grpndw[outofsample]=-99.
resbg3grpradeg[outofsample]=-99.
resbg3grpdedeg[outofsample]=-99.
resbg3grpcz[outofsample]=-99.
resbg3rproj[outofsample]=-99.
resbg3router[outofsample]=-99.
resbg3fc[outofsample]=-99.
resbg3vdisp[outofsample]=-99.

Lastly, we need to extract the RESOLVE-A groups from ECO, and unify the RESOLVE group catalog. The easiest way to do this is to populate empty arrays, assigning values to individual indices based on the RESOLVE name (`rsXXXX`=RESOLVE-A, `rfXXXX`=RESOLVE-B).

In [ ]:
# make empty arrays to hold A- and B-semester information
sz = len(resall)
resolvename = np.array(resall['name'])
resolveg3grp = np.full(sz, -99.)
resolveg3grpngi = np.full(sz, -99.)
resolveg3grpndw = np.full(sz, -99.)
resolveg3grpradeg = np.full(sz, -99.)
resolveg3grpdedeg = np.full(sz, -99.)
resolveg3grpcz = np.full(sz, -99.)
resolveg3rproj = np.full(sz,-99.)
resolveg3fc = np.full(sz,-99.)
resolveg3router = np.full(sz,-99.)
resolveg3vdisp = np.full(sz,-99.)

for i,nm in enumerate(resolvename):
    if nm.startswith('rs'):
        # if name starts with 'rs', the galaxy is in RESOLVE-A, so draw group info from ECO
        sel_in_eco = np.where(ecoresname==nm)
        resolveg3grp[i] = ecog3grp[sel_in_eco]
        resolveg3grpngi[i] = ecog3grpngi[sel_in_eco]
        resolveg3grpndw[i] = ecog3grpndw[sel_in_eco]
        resolveg3grpradeg[i] = ecog3grpradeg[sel_in_eco]
        resolveg3grpdedeg[i] = ecog3grpdedeg[sel_in_eco]
        resolveg3grpcz[i] = ecog3grpcz[sel_in_eco]
        resolveg3rproj[i] = ecog3rproj[sel_in_eco]
        resolveg3fc[i] = ecog3fc[sel_in_eco]
        resolveg3router[i]=ecog3router[sel_in_eco]
        resolveg3vdisp[i]=ecog3vdisp[sel_in_eco]
    elif nm.startswith('rf'):
        # if name starts with 'rf', use separately-computed RESOLVE-B group info
        sel_in_resb = np.where(resbname==nm)
        resolveg3grp[i] = resbg3grp[sel_in_resb]
        resolveg3grpngi[i] = resbg3grpngi[sel_in_resb]
        resolveg3grpndw[i] = resbg3grpndw[sel_in_resb]
        resolveg3grpradeg[i] = resbg3grpradeg[sel_in_resb]
        resolveg3grpdedeg[i] = resbg3grpdedeg[sel_in_resb]
        resolveg3grpcz[i] = resbg3grpcz[sel_in_resb]
        resolveg3rproj[i] = resbg3rproj[sel_in_resb]
        resolveg3fc[i] = resbg3fc[sel_in_resb]
        resolveg3router[i] = resbg3router[sel_in_resb]
        resolveg3vdisp[i] = resbg3vdisp[sel_in_resb]
    else:
        assert False, nm+" not in RESOLVE"
        
# assign quantities to dataframe
resall['g3grp_l']=resolveg3grp
resall['g3grpngi_l']=resolveg3grpngi
resall['g3grpndw_l']=resolveg3grpndw
resall['g3grpradeg_l']=resolveg3grpradeg
resall['g3grpdedeg_l']=resolveg3grpdedeg
resall['g3grpcz_l']=resolveg3grpcz
resall['g3rproj_l']=resolveg3rproj
resall['g3fc_l']=resolveg3fc
resall['g3router_l']=resolveg3router
resall['g3vdisp_l']=resolveg3vdisp

## Examples

In [ ]:
ecosurvey = ecoall[np.where((ecoall['absrmag']<=-17.33) & (ecoall['g3grpcz_l']>3000.) & (ecoall['g3grpcz_l']<7000.))]
resolvesurvey = resall[np.where((resall['fl_insample']) & (resall['g3grpcz_l']>4500) & (resall['g3grpcz_l']<7000))]

### (i) Group Radii Plotting

In this first example, let's look at the largest ECO cluster identified using the G3 algorithm. The cell below selects this system and plots its members and several reference radii: the virial radius (r337b), the 75th-percentile observational projected radius, and the outermost group member radius. If you set `ADAPTIVE_LINKING=1` earlier in this tutorial, you will notice that the clusters members are often extended far beyond the virial radius. The adaptive linking strategy, especially at the high masses, tends to finds group members at 2-4$R_{\rm vir}$ in the splashback/infall regions. On the other hand, if you set `ADAPTIVE_LINKING=0`, then nearly-all group members will fall within $1R_{\rm vir}$. You may wish to re-run this tutorial and see how the results change if you change the adaptive option.

*Note*: If any of the radii in the plot below have apparently unphysical values (e.g., $10^{20}$ degrees), you may have run the iPython cells out of order. Try restarting the Jupyter kernel and running all cells in order (`Kernel` > `Restart and Run All`).

In [ ]:
largecluster = ecosurvey[np.where(ecosurvey['g3grpngi_l']==np.max(ecosurvey['g3grpngi_l']))]

In [ ]:
def circle(xcen,ycen,r):
    theta = np.linspace(0,2*np.pi,720)
    return xcen+r*np.cos(theta), ycen+r*np.sin(theta)

plt.figure(figsize=(8,8))
plt.plot(largecluster['radeg'], largecluster['dedeg'], 'k.', label='Cluster Galaxies', alpha=0.7)
i = 0
plt.plot(*circle(largecluster['g3grpradeg_l'][i], largecluster['g3grpdedeg_l'][i],\
                 largecluster['g3rvir_l'][i]*0.000277778), label='Virial Radius (r337b)', color='purple')
plt.plot(*circle(largecluster['g3grpradeg_l'][i], largecluster['g3grpdedeg_l'][i],\
                 largecluster['g3rproj_l'][i]*0.000277778), label='75% Projected Radius')
plt.plot(*circle(largecluster['g3grpradeg_l'][i], largecluster['g3grpdedeg_l'][i],\
                 largecluster['g3router_l'][i]*0.000277778), label='Outermost Galaxy Radius')
plt.plot(largecluster['g3grpradeg_l'][i], largecluster['g3grpdedeg_l'][i], 'rx', label='Group Center', markersize=10)
plt.xlabel("RA [deg]")
plt.ylabel("Dec [deg]")
plt.legend(loc='best', framealpha=1)
plt.show()

### (ii) Gas-to-Stellar Mass Ratio and Halo Mass
As another simple example, we can look at the relationship between gas fraction and group halo mass. The cells below compute the gas fractions and make the plot.

In [ ]:
res_gs = (10.**resolvesurvey['logmgas'])/(10.**resolvesurvey['logmstar'])
eco_gs = (10.**ecosurvey['logmgas'])/(10.**ecosurvey['logmstar'])

In [ ]:
plt.figure(figsize=(13,6))

# eco has a couple missing logmstar/logmgas - remove these for clarity
sel = np.where((ecosurvey['logmstar']>0))

plt.plot(ecosurvey['g3logmh_l'][sel], eco_gs[sel], '.', color='lightblue', alpha=0.2)
plt.plot(resolvesurvey['g3logmh_l'], res_gs, '.', color='lightcoral', alpha=0.4)

medecog3gs, binedges1, junk = binned_statistic(ecosurvey['g3logmh_l'][sel], eco_gs[sel], statistic='median', bins=15)
medresg3gs, binedges2, junk = binned_statistic(resolvesurvey['g3logmh_l'], res_gs, statistic='median', bins=12)


plt.plot(binedges1[:-1], medecog3gs,'^-', color='darkblue', label='ECO')
plt.plot(binedges2[:-1], medresg3gs,'^-', color='darkred', label='RESOLVE')
plt.legend(loc='best', framealpha=1)
plt.yscale('log')
plt.ylim(1e-2,1e2)
plt.xlim(10.5,15)
plt.xlabel("log Group Halo Mass")
plt.ylabel("Galaxy Gas-to-Stellar Mass Ratio")
plt.show()